# Evaluate using Azure OpenAI Graders with Azure AI Foundry SDK

## Objective

This tutorial offers a step-by-step guide for evaluating large language models using Azure OpenAI Graders and their model outputs.
In Azure AI Foundry SDK, we are now supporting four new AOAI Graders:
- Model Labeler: Uses your custom prompt to instruct a model to classify outputs based on labels you define. It returns structured results with explanations for why each label was chosen.
- String Check: Compares input text to a reference value, checking for exact or partial matches with optional case insensitivity. Useful for flexible text validations and pattern matching.
- Text Similarity: Evaluates how closely input text matches a reference value using similarity metrics like`fuzzy_match`, `BLEU`, `ROUGE`, or `METEOR`. Useful for assessing text quality or semantic closeness.
- General Grader: Advanced users have the capability to import or define a custom grader and integrate it into the AOAI general grader. This allows for evaluations to be performed based on specific areas of interest aside from the existing AOAI graders. 

This tutorial uses the following AI services:
- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend about 15 minutes running this notebook.

## Before you begin

### Installation

Install the following packages requried to execute this notebook.

In [ ]:
%pip install azure.ai.projects
%pip install azure_ai_evaluation

### Parameters and imports

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
from azure.ai.evaluation import (
    AzureOpenAIModelConfiguration,
    AzureOpenAILabelGrader,
    AzureOpenAIStringCheckGrader,
    AzureOpenAITextSimilarityGrader,
    AzureOpenAIGrader,
    evaluate,
)
from openai.types.eval_string_check_grader import EvalStringCheckGrader

### Environment variables

Set these environment variables with your own values:
1) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
2) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
3) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.
4) **MODEL_DEPLOYMENT_NAME** - Model deployment to be used for evaluation
5) **AZURE_SUBSCRIPTION_ID** - Azure Subscription Id of Azure AI Project
6) **PROJECT_NAME** - Azure AI Project Name
7) **RESOURCE_GROUP_NAME** - Azure AI Project Resource Group Name

In [ ]:
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)

project = {
    "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
    "project_name": os.environ["PROJECT_NAME"],
    "resource_group_name": os.environ["RESOURCE_GROUP_NAME"],
}

### Data
To run the evaluation against your own data, replace this data file with your sample data

In [ ]:
fname = "data.jsonl"

## Create grader objects

Before executing the evaluation, the grader objects needs to be defined. Aside from `model_config` and customizable grader `name`, each graders have few unique parameters that are required for set up.

### Model Labeler

This grader uses your custom prompt to instruct a model to classify outputs based on labels you define. It returns structured results with explanations for why each label was chosen. To correctly set up, the following unique parameters are required:
- input: Identifies the column in `data.jsonl` that the evaluator will use for classification. It also defines the custom prompt that instructs the model on how to perform the classification.
- labels: Lists all possible labels that the evaluator can assign to the input data based on the custom prompt.
- passing_labels: Specifies which of the defined labels are considered successful or acceptable outcomes.
- model: Indicates the model that will be used to classify the input data according to the custom prompt.

In [ ]:
# Determine if the response column contains texts that are too short, just right, or too long
label_grader = AzureOpenAILabelGrader(
    model_config=model_config,
    input=[
        {"content": "{{item.response}}", "role": "user"},
        {
            "content": "Any text including space that's more than 600 characters are too long, less than 500 characters are too short; 500 to 600 characters are just right.",
            "role": "user",
            "type": "message",
        },
    ],
    labels=["too short", "just right", "too long"],
    passing_labels=["just right"],
    model="gpt-4o",
    name="label",
)

### Text Similarity

This grader evaluates how closely input text matches a reference value using similarity metrics like`fuzzy_match`, `BLEU`, `ROUGE`, or `METEOR`. Useful for assessing text quality or semantic closeness. To correctly set up, the following unique parameters are required:
- evaluation_metric: Specifies the similarity metric to be used for evaluation.
- input: Specifies the column in `data.jsonl` that that contains the text to be evaluated.
- pass_threshold: Defines the minimum similarity score required for an output to be considered a passing result in the evaluation.
- reference: specifies the column in `data.jsonl` that contains the reference text against which the input will be compared.

In [ ]:
# Pass if response column and ground_truth column similarity score >= 0.5 using "fuzzy_match"
sim_grader = AzureOpenAITextSimilarityGrader(
    model_config=model_config,
    evaluation_metric="fuzzy_match",  # support evaluation metrics including: "fuzzy_match", "bleu", "gleu", "meteor", "rouge_1", "rouge_2", "rouge_3", "rouge_4", "rouge_5", "rouge_l", "cosine".
    input="{{item.response}}",
    name="similarity",
    pass_threshold=0.5,
    reference="{{item.ground_truth}}",
)

### String Check

This grader compares input text to a reference value, checking for exact or partial matches with optional case insensitivity. Useful for flexible text validations and pattern matching. To correctly set up, the following unique parameters are required:
- input: Specifies the column in `data.jsonl` that that contains the text to be evaluated.
- operation: Defines the operation type of this grader object.
- reference: specifies the reference value against which the input will be compared.

In [ ]:
# Pass if the query column contains "What is"
string_grader = AzureOpenAIStringCheckGrader(
    model_config=model_config,
    input="{{item.query}}",
    name="Contains What is",
    operation="like",  # "eq" for equal, "ne" for not equal, "like" for contain, "ilike" for case insensitive contain
    reference="What is",
)

### General Grader

This grader enables advanced users to import or define a custom grader and integrate it into the AOAI general grader. This allows for evaluations to be performed based on specific areas of interest aside from the existing AOAI graders. To correctly set up, a separate grader object needs to be created and defined. The defined grader can be passed into the general grader using parameter `grader_config`.

In [ ]:
# Define an string check grader config directly using the OAI SDK
oai_string_check_grader = EvalStringCheckGrader(
    input="{{item.query}}", name="contains hello", operation="like", reference="hello", type="string_check"
)
# Plug that into the general grader
general_grader = AzureOpenAIGrader(model_config=model_config, grader_config=oai_string_check_grader)

## Evaluation

Once all the grader objects have been correctly set up, we can evaluate the test dataset against the graders using the `evaluate` method. Optionally, you may add `azure_ai_project=project` in the evaluate call to upload the evaluation result to your Foundry project.

In [ ]:
evaluation = evaluate(
    data=fname,
    evaluators={
        "label": label_grader,
        "general": general_grader,
        "string": string_grader,
        "similarity": sim_grader,
    },
    # azure_ai_project=project
)
evaluation